In [ ]:
import requests
from bs4 import BeautifulSoup
import copy
import csv
from tqdm import tqdm

In [ ]:
topic_url = 'https://viterotiles.com/'
response = requests.get(topic_url)

In [ ]:
response.status_code

200

In [ ]:
page_contents = response.text
doc = BeautifulSoup(page_contents, 'html.parser')

## Getting the above the value text

In [ ]:
mydivs = doc.find(class_="tile-gallery grey-block main-block-row").find_all('a',href=True)

## Get the values of category

In [ ]:
s="https://viterotiles.com"
j=[]
for  a in mydivs:
    j.append(s+a['href'])
print(j)

['https://viterotiles.com/living-room-tiles', 'https://viterotiles.com/outdoor-tiles', 'https://viterotiles.com/commercial-tiles', 'https://viterotiles.com/bathroom-tiles', 'https://viterotiles.com/kitchen-tiles', 'https://viterotiles.com/bedroom-tiles']


## This is for coping the Beautiful Soup Objects

In [ ]:
from bs4 import Tag, NavigableString
def clone(el):
    if isinstance(el, NavigableString):
        return type(el)(el)
    copy = Tag(None, el.builder, el.name, el.namespace, el.nsprefix)
    # work around bug where there is no builder set
    # https://bugs.launchpad.net/beautifulsoup/+bug/1307471
    copy.attrs = dict(el.attrs)
    for attr in ('can_be_empty_element', 'hidden'):
        setattr(copy, attr, getattr(el, attr))
    for child in el.contents:
        copy.append(clone(child))
    return copy

## Main Code Getting Data Form the ViteroTile Website

In [ ]:
from re import A
lis=[]
for a in tqdm(j):
    variant_application=[]
    if a=='https://viterotiles.com/living-room-tiles':
      variant_application.append('Living_Room')
    elif a=='https://viterotiles.com/outdoor-tiles':
      variant_application.append('OutDoor_Tiles')
    elif a=='https://viterotiles.com/commercial-tiles':
      variant_application.append('Commercial_Tiles')
    elif a=='https://viterotiles.com/bathroom-tiles':
      variant_application.append('Bathroom_Tile')
    elif a=='https://viterotiles.com/kitchen-tiles':
      variant_application.append('Kithen_Tiles')
    elif a=='https://viterotiles.com/bedroom-tiles':
      variant_application.append('Bedroom_Tiles')
    response = requests.get(a)
    page_contents = response.text
    doc = BeautifulSoup(page_contents, 'html.parser')
    x=doc.find(class_="list-items").find_all('a',href=True)
    s="https://viterotiles.com"
    d=[]
    for f in x:
        if not(f['href'] == "javascript:void(0);"):
            d.append(s+f['href'])
    for k in d:
        response = requests.get(k)
        page_contents1 = response.text
        doc3 = BeautifulSoup(page_contents1, 'html.parser')
        xy=doc3.find_all(class_="product-container")
        bs=[]
        count=0
        for div in doc3:
            cp=clone(div)
            if '<div class="info">' in str(cp) or count!=0:
                count=count+1
                if count==15:
                    break
                bs.append(div)
        g=[]
        h=[]
        dictionary={}
        for ij in range(len(bs)):
            if isinstance(bs[ij], NavigableString):
                continue
            for gh,hk in zip(bs[ij].find_all('span',{"class":"name"}),bs[ij].find_all('span',{"class":"value"})):
                dictionary[gh.text]=hk.text
        for key, value in dictionary.items():
            val=[]
            c=(value.replace('\n',"").replace('\t',"").replace(',',"").strip())
            fg=c.split("|")
            for jkl in fg:
                jkl=jkl.strip()
                val.append(jkl)
            dictionary[key]=val
        if dictionary!={}:
                dictionary['Variant_Application']=[]
                dictionary['Variant_Application'].extend(variant_application)
        lis.append(dictionary)

100%|██████████| 6/6 [35:27<00:00, 354.50s/it]


In [67]:
len(lis)

915

In [68]:
while {} in lis:
  lis.remove({})

In [69]:
len(lis)

915

In [ ]:
import pickle
# # Create a variable
# myvar = [{'This': 'is', 'Example': 2}, 'of',
#          'serialisation', ['using', 'pickle']]
# Open a file and use dump()
with open('list.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(lis, file)

In [ ]:
import pickle
# Open the file in binary mode
with open('list.pkl', 'rb') as file:
    # Call load method to deserialze
    lis = pickle.load(file)
    # print(myvar)

In [ ]:
lis[0]

{'Avg. Weight (Kgs)': ['30', '42', '43'],
 'Category': ['Double Charged Vitrified Tiles'],
 'Color': ['Cream'],
 'Coverage Area (Sq.mt)': ['1.44', '1.92', '1.92'],
 'Finish': ['Glossy'],
 'Qty. Per Box (Pcs)': ['4', '3', '2'],
 'Sizes (mm)': ['600 x 600', '800 x 800', ''],
 'Suitability': ['Floor'],
 'Variant_Application': ['Living_Room']}

In [70]:
csv_columns = ['Avg. Weight (Kgs)','Suitability','Category','Color','Finish','Sizes (mm)','Qty. Per Box (Pcs)','Coverage Area (Sq.mt)','Variant_Application']
dict_data = lis
csv_file = "Tiles_preprocessed.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in lis:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [71]:
import pandas as pd
df=pd.read_csv('Tiles_preprocessed.csv')

In [ ]:
lis[466]

{'Avg. Weight (Kgs)': ['30'],
 'Category': ['Glazed Vitrified Tiles'],
 'Color': ['Grey'],
 'Coverage Area (Sq.mt)': ['1.44'],
 'Finish': ['Glossy                                                                                                                             Matt'],
 'Qty. Per Box (Pcs)': ['4'],
 'Sizes (mm)': ['600 x 600'],
 'Suitability': ['Floor and Wall'],
 'Variant_Application': ['Bathroom_Tile']}

In [72]:
df

,Avg. Weight (Kgs),Suitability,Category,Color,Finish,Sizes (mm),Qty. Per Box (Pcs),Coverage Area (Sq.mt),Variant_Application
0,"['30', '42', '43']",['Floor'],['Double Charged Vitrified Tiles'],['Cream'],['Glossy'],"['600 x 600', '800 x 800', '']","['4', '3', '2']","['1.44', '1.92', '1.92']",['Living_Room']
1,"['30', '42', '43']",['Floor'],['Double Charged Vitrified Tiles'],['White'],['Glossy'],"['600 x 600', '800 x 800', '']","['4', '3', '2']","['1.44', '1.92', '1.92']",['Living_Room']
2,['30'],['Floor'],['Double Charged Vitrified Tiles'],['Black'],['Glossy'],['600 x 600'],['4'],['1.44'],['Living_Room']
3,['30'],['Floor'],['Double Charged Vitrified Tiles'],['Brown'],['Glossy'],['600 x 600'],['4'],['1.44'],['Living_Room']
4,['30'],['Floor'],['Double Charged Vitrified Tiles'],['Brown'],['Glossy'],['600 x 600'],['4'],['1.44'],['Living_Room']
...,...,...,...,...,...,...,...,...,...
910,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['Cream'],['Carving'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
911,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['White'],['Carving'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
912,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['Cream'],['Carving'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
913,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['White'],['Glossy'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']


## The duplicated Values are below

In [73]:
duplicate = df[df.duplicated()]
duplicate


,Avg. Weight (Kgs),Suitability,Category,Color,Finish,Sizes (mm),Qty. Per Box (Pcs),Coverage Area (Sq.mt),Variant_Application
4,['30'],['Floor'],['Double Charged Vitrified Tiles'],['Brown'],['Glossy'],['600 x 600'],['4'],['1.44'],['Living_Room']
6,['30'],['Floor'],['Double Charged Vitrified Tiles'],['Grey'],['Glossy'],['600 x 600'],['4'],['1.44'],['Living_Room']
9,['30'],['Floor'],['Double Charged Vitrified Tiles'],['Brown'],['Glossy'],['600 x 600'],['4'],['1.44'],['Living_Room']
10,"['30', '42', '43']",['Floor'],['Double Charged Vitrified Tiles'],['White'],['Glossy'],"['600 x 600', '800 x 800', '']","['4', '3', '2']","['1.44', '1.92', '1.92']",['Living_Room']
12,['42'],['Floor'],['Double Charged Vitrified Tiles'],['Black'],['Glossy'],['800 x 800'],['3'],['1.92'],['Living_Room']
...,...,...,...,...,...,...,...,...,...
908,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['Black'],['High Glossy'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
909,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['Black'],['High Glossy'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
911,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['White'],['Carving'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
912,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['Cream'],['Carving'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']


## Remove Duplicate File

In [74]:
df.drop_duplicates(keep=False,inplace=True)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228 entries, 2 to 914
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Avg. Weight (Kgs)      228 non-null    object
 1   Suitability            228 non-null    object
 2   Category               228 non-null    object
 3   Color                  228 non-null    object
 4   Finish                 228 non-null    object
 5   Sizes (mm)             228 non-null    object
 6   Qty. Per Box (Pcs)     228 non-null    object
 7   Coverage Area (Sq.mt)  228 non-null    object
 8   Variant_Application    228 non-null    object
dtypes: object(9)
memory usage: 17.8+ KB


## The final Dataframe

In [76]:
df

,Avg. Weight (Kgs),Suitability,Category,Color,Finish,Sizes (mm),Qty. Per Box (Pcs),Coverage Area (Sq.mt),Variant_Application
2,['30'],['Floor'],['Double Charged Vitrified Tiles'],['Black'],['Glossy'],['600 x 600'],['4'],['1.44'],['Living_Room']
7,"['30', '42']",['Floor'],['Double Charged Vitrified Tiles'],['Black'],['Glossy'],"['600 x 600', '800 x 800']","['4', '3']","['1.44', '1.92']",['Living_Room']
8,['42'],['Floor'],['Double Charged Vitrified Tiles'],['Grey'],['Glossy'],['800 x 800'],['3'],['1.92'],['Living_Room']
15,"['42', '43']",['Floor'],['Double Charged Vitrified Tiles'],['White'],['Glossy'],"['800 x 800', '800 x 1200']","['3', '2']","['1.92', '1.92']",['Living_Room']
18,"['30', '42']",['Floor'],['Double Charged Vitrified Tiles'],['Grey'],['Glossy'],"['600 x 600', '800 x 800']","['4', '3']","['1.44', '1.92']",['Living_Room']
...,...,...,...,...,...,...,...,...,...
880,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['Cream'],['High Glossy'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
881,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['Grey'],['High Glossy'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
905,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['Green'],['Glossy'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
906,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['White'],['Glossy ...,['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']


In [77]:
df.to_csv('Tiles_preprocessed.csv')

In [80]:
df=pd.read_csv('Tiles_preprocessed.csv')

In [81]:
df

,Unnamed: 0,Avg. Weight (Kgs),Suitability,Category,Color,Finish,Sizes (mm),Qty. Per Box (Pcs),Coverage Area (Sq.mt),Variant_Application
0,2,['30'],['Floor'],['Double Charged Vitrified Tiles'],['Black'],['Glossy'],['600 x 600'],['4'],['1.44'],['Living_Room']
1,7,"['30', '42']",['Floor'],['Double Charged Vitrified Tiles'],['Black'],['Glossy'],"['600 x 600', '800 x 800']","['4', '3']","['1.44', '1.92']",['Living_Room']
2,8,['42'],['Floor'],['Double Charged Vitrified Tiles'],['Grey'],['Glossy'],['800 x 800'],['3'],['1.92'],['Living_Room']
3,15,"['42', '43']",['Floor'],['Double Charged Vitrified Tiles'],['White'],['Glossy'],"['800 x 800', '800 x 1200']","['3', '2']","['1.92', '1.92']",['Living_Room']
4,18,"['30', '42']",['Floor'],['Double Charged Vitrified Tiles'],['Grey'],['Glossy'],"['600 x 600', '800 x 800']","['4', '3']","['1.44', '1.92']",['Living_Room']
...,...,...,...,...,...,...,...,...,...,...
223,880,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['Cream'],['High Glossy'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
224,881,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['Grey'],['High Glossy'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
225,905,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['Green'],['Glossy'],['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
226,906,['32'],['Floor and Wall'],['Glazed Vitrified Tiles'],['White'],['Glossy ...,['600 x 1200'],['2'],['1.44'],['Bedroom_Tiles']
